In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import math


Let us consider a simple function which where we compute the maximum of two vectors and return the value in the third

In [ ]:
# Initializing the random vectors to compute the maximum
a = np.random.rand(32,32,32).astype(np.float32)
b = np.random.rand(32,32,32).astype(np.float32)
c = np.zeros_like(a)

The most naive way to compute the maximum would look somewhat like this

In [ ]:
def maximum_1(a,b,c):
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            for k in range(a.shape[2]):
                c[i,j,k] = max(a[i,j,k],b[i,j,k])
    return c
%timeit maximum_1(a,b,c)


100 loops, best of 3: 19.6 ms per loop


However, numpy has a function [np.maximum](https://numpy.org/doc/stable/reference/generated/numpy.maximum.html), which performs the same task. Let us time it and compare the performance.

In [ ]:

def maximum_2(a,b,c):
    c = np.maximum(a,b)
    return c
%timeit maximum_2(a,b,c)

The slowest run took 4.01 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 172 µs per loop


Now, there is a 100X improvement in speed of execution of  maximum_1 and maximum_2 methods. **Why?**

# Part 1: Introducing Numba

Python is a interpreted language and not a compiled language. numba provides a just-in-time compiler which allows you to run the code very quickly.



In [ ]:
! pip list
from numba import jit, njit, vectorize, cuda

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.2.10         
astor                         0.8.1          
astropy                       4.1            
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.2.0         
audioread                     2.1.9          
autograd                      1.3            
Babel                         2.8.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bleach                        3.2.1          
blis                          0.4.

In [ ]:
@jit(nopython=True)
def maximum_3(a,b,c):
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            for k in range(a.shape[2]):
                c[i,j,k] = max(a[i,j,k],b[i,j,k])
    return c
# c = maximum_3(a,b,c)
# c = maximum_3(a,b,c)
%timeit maximum_3(a,b,c)

The slowest run took 19602.14 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 8.86 µs per loop


Another way to use numba to accelerate is vectorize

In [ ]:
@vectorize(['float32(float32,float32)'])
def maximum_4(a,b):
  return max(a,b)

%timeit c = maximum_4(a,b)

The slowest run took 32.21 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.97 µs per loop


# Part 2: Numba + CUDA on Google Colab
By default, Google Colab is not able to run numba + CUDA, because two lilbraries are not found, libdevice and libnvvm.so. So we need to make sure that these libraries are found in the notebook.

First, we look for these libraries on the system. To do that, we simply run the find command, to recursively look for these libraries starting at the root of the filesystem. The exclamation mark escapes the line so that it's executed by the Linux shell, and not by the jupyter notebook.

In [ ]:
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'

/usr/local/cuda-10.0/nvvm/libdevice
/usr/local/cuda-10.1/nvvm/libdevice
/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so
/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so



Then, we add the two libraries to numba environment variables:

In [ ]:
import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-10.1/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so"

In [ ]:
@vectorize(['float32(float32,float32)'],target='cuda')
def maximum_5(a,b):
  return max(a,b)

%timeit c = maximum_5(a,b)

The slowest run took 103.01 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.1 ms per loop


Wait! We do not gain anything and the CPU version is actually twice faster! 

There is a simple reason for this. When running on the GPU, the following happens under the hood: 

* the input data is transferred to the GPU memory;
* the calculation of the maximum is done in parallel on the GPU for all elements;
* the resulting array is sent back to the host system. 

If the calculation is too simple, there is no use shipping our data to the GPU for fast parallel processing, if we are to wait so long for the data transfers to complete. In other words, most of the time is spent in the data transfers, and the GPU is basically useless.

Let's see what happens with a more involved calculation. 



In [ ]:
a = np.random.rand(128,128,128).astype(np.float32)
b = np.random.rand(128,128,128).astype(np.float32)
c = np.zeros_like(a)

In [ ]:
%timeit c = maximum_4(a,b)
%timeit c = maximum_5(a,b)

100 loops, best of 3: 7.95 ms per loop
100 loops, best of 3: 7.92 ms per loop


In [ ]:
a = np.random.rand(256,256,256).astype(np.float32)
b = np.random.rand(256,256,256).astype(np.float32)
c = np.zeros_like(a)

In [ ]:
%timeit c = maximum_4(a,b)
%timeit c = maximum_5(a,b)

10 loops, best of 3: 48.1 ms per loop
10 loops, best of 3: 47.5 ms per loop


# Part 3: GPU programming in python using numba

### Writing Cuda Kernels

While targeting ufuncs with the `cuda` syntax is the most straightforward way to access the GPU with Numba, it may not be flexible enough for your needs. If you want to write a more detailed GPU program, at some point you are probably going to need to write CUDA kernels.

As discussed in the lecture, the CUDA programming model allows you to abstract the GPU hardware into a software model composed of a **grid** containing **blocks** of **threads**. These threads are the smallest individual unit in the programming model, and they execute together in groups (traditionally called **warps**, consisting of 32 threads each). Determiming the best size for your grid of thread blocks is a complicated problem that often depends on the specific algorithm and hardware you're using, but here a few good rules of thumb:
+ the size of a block should be a multiple of 32 threads, with typical block sizes between 128 and 512 threads per block.
+ the size of the grid should ensure the full GPU is utilized where possible. Launching a grid where the number of blocks is 2x-4x the number of **streaming multiprocessors** on the GPU is a good starting place. (The Tesla K80 GPUs provided by Colaboratory have 15 SMs - more modern GPUs like the P100s on TigerGPU have 60+.)
+ The CUDA kernel launch overhead does depend on the number of blocks, so it may not be best to launch a grid where the number of threads equals the number of input elements when the input size is very big. We'll show a pattern for dealing with large inputs below.

As a first example, let's return to our vector addition function, but this time, we'll target it with the `cuda.jit` decorator:

In [ ]:
@cuda.jit
def maximum_6(a, b, c):
    tidx = cuda.threadIdx.x 
    bidx = cuda.blockIdx.x  
    tidy = cuda.threadIdx.y 
    bidy = cuda.blockIdx.y  
    tidz = cuda.threadIdx.z 
    bidz = cuda.blockIdx.z  


    block_dimx = cuda.blockDim.x  # number of threads per block
    grid_dimx = cuda.gridDim.x    # number of blocks in the grid
    block_dimy = cuda.blockDim.y  # number of threads per block
    grid_dimy = cuda.gridDim.y    # number of blocks in the grid
    block_dimz = cuda.blockDim.z  # number of threads per block
    grid_dimz = cuda.gridDim.z    # number of blocks in the grid

    idx = block_dimx*bidx + tidx
    idy = block_dimy*bidy + tidy
    idz = block_dimz*bidz + tidz
    c[idx,idy,idz] = max(a[idx,idy,idz],b[idx,idy,idz])

In [ ]:
threads_per_block = (8, 8, 8)
blocks_per_grid = (32, 32, 32)
a_device = cuda.to_device(a)
b_device = cuda.to_device(b)
c_device = cuda.to_device(c)
# Note, the indexing should account for total number of elements
%timeit maximum_6[blocks_per_grid, threads_per_block](a_device, b_device, c_device)
c_device = c_device.copy_to_host()

1000 loops, best of 3: 1.76 ms per loop
